In [2]:
# ---------------------------------------------------------------
#  LQ fits to σ-LEM model curves
#  (0 / 0.25 / 0.50 / 1.00 mM AuNP, 100 kVp)
#  First-order curves include the Z ≥ –1/σ truncation factor
# ---------------------------------------------------------------
import numpy as np
import pandas as pd
from math import erfc
v_erfc = np.vectorize(erfc)

# ---------- σ-LEM fixed parameters ------------------------------
alpha_ctrl, beta_ctrl = 0.0252, 0.00130      # bare BAEC
Kc          = 2.11                           # 100 kVp beam

# ---------- Dose grid -------------------------------------------
D = np.arange(0.0, 6.01, 0.25)               # 0 … 6 Gy, 0.25-Gy steps

# ---------- Helper functions ------------------------------------
def sigma(c):                                # log-normal width
    return np.sqrt(2 * np.log1p(Kc * c))

def _trunc_factor(d, s):
    A = np.sqrt(1 + 2 * beta_ctrl * s**2 * d**2)
    shift = (-1 / s) + (s * d * (alpha_ctrl + 2 * beta_ctrl * d)) / (1 + 2 * beta_ctrl * s**2 * d**2)
    return v_erfc(A / np.sqrt(2) * shift) / erfc(-1 / (s * np.sqrt(2)))

# ---------- σ-LEM hierarchy -------------------------------------
def S_LQ(d):                      # baseline (0 mM)
    return np.exp(-alpha_ctrl * d - beta_ctrl * d**2)

def S_var_trunc(d, c):            # first-order, variance-only + trunc
    s   = sigma(c)
    Den = 1 + 2 * beta_ctrl * s**2 * d**2
    Num = -alpha_ctrl * d - beta_ctrl * d**2 + (alpha_ctrl**2 * s**2 * d**2) / (2 * Den)
    return np.exp(Num) / np.sqrt(Den) * _trunc_factor(d, s)

def S_mix_trunc(d, c):            # first-order, mixed (α→α⋆) + trunc
    s   = sigma(c)
    a_  = alpha_ctrl + 2 * beta_ctrl * d
    Den = 1 + 2 * beta_ctrl * s**2 * d**2
    Num = -alpha_ctrl * d - beta_ctrl * d**2 + (a_ * s * d)**2 / (2 * Den)
    return np.exp(Num) / np.sqrt(Den) * _trunc_factor(d, s)

def S_second(d, c):               # complete second-order
    s2 = sigma(c)**2
    Den = 1 + s2 * d * (alpha_ctrl + 4 * beta_ctrl * d)
    Num = -alpha_ctrl * d - beta_ctrl * d**2 + s2 * d**2 * (alpha_ctrl + 2 * beta_ctrl * d)**2 / (2 * Den)
    return np.exp(Num) / np.sqrt(Den)

model = {0.00: lambda d,c: S_LQ(d),
         0.25: lambda d,c: S_mix_trunc(d,c),
         0.50: lambda d,c: S_mix_trunc(d,c),
         1.00: lambda d,c: S_second(d,c)}

# ---------- plain (un-weighted) LQ fit on ln S -------------------
def fit_LQ_unweighted(S_vals):
    """
    ln S ≈ –(α' D + β' D²),  ordinary least-squares, 0-Gy row excluded.
    Returns α', β', their 1-σ errors, and R².
    """
    mask = D > 0                         # drop 0-Gy row
    y = np.log(S_vals[mask])
    X = np.vstack((D[mask], D[mask]**2)).T
    coef, *_ = np.linalg.lstsq(X, -y, rcond=None)
    alpha_p, beta_p = coef

    # residuals and R²
    y_hat = -(alpha_p * D[mask] + beta_p * D[mask]**2)
    ss_res = np.sum((y - y_hat)**2)
    ss_tot = np.sum((y - y.mean())**2)
    R2 = 1 - ss_res / ss_tot

    # covariance estimate
    dof = len(y) - 2
    sigma2 = ss_res / dof
    cov = sigma2 * np.linalg.inv(X.T @ X)
    se_alpha, se_beta = np.sqrt(np.diag(cov))

    return alpha_p, beta_p, se_alpha, se_beta, R2

# ---------- Run fits ---------------------------------------------
records = []
for conc in (0.00, 0.25, 0.50, 1.00):
    S_curve = model[conc](D, conc)
    a_p, b_p, se_a, se_b, r2 = fit_LQ_unweighted(S_curve)
    records.append(dict(conc_mM=conc,
                        alpha_prime=a_p, sigma_alpha=se_a,
                        beta_prime=b_p,  sigma_beta=se_b,
                        R2=r2))

fit_df = pd.DataFrame(records)

# ---------- Show & save ------------------------------------------
pd.options.display.float_format = "{:0.4e}".format
print("\nOrdinary  LQ fit to σ-LEM model curves")
print("Model mapping:")
print("  0.00 mM : baseline LQ")
print("  0.25 mM : first-order mixed term  (truncated)")
print("  0.50 mM : first-order mixed term     (truncated)")
print("  1.00 mM : complete second-order")
print("\n" + "="*80)
print(fit_df.to_string(index=False))

fit_df.to_csv("sigmaLEM_LQ_fits_unweighted.csv", index=False)
print("\nResults saved to  sigmaLEM_LQ_fits_unweighted.csv")



Ordinary (un-weighted) LQ fit to σ-LEM model curves
Model mapping:
  0.00 mM : baseline LQ
  0.25 mM : first-order mixed term  (truncated)
  0.50 mM : first-order mixed term     (truncated)
  1.00 mM : complete second-order

   conc_mM  alpha_prime  sigma_alpha  beta_prime  sigma_beta         R2
0.0000e+00   2.5200e-02   1.0888e-17  1.3000e-03  2.2960e-18 1.0000e+00
2.5000e-01   3.2307e-02   1.1471e-04  1.9492e-03  2.4190e-05 9.9996e-01
5.0000e-01   3.7987e-02   2.0572e-04  2.3938e-03  4.3381e-05 9.9991e-01
1.0000e+00   5.9030e-02   4.3109e-04  2.6037e-03  9.0908e-05 9.9981e-01

Results saved to  sigmaLEM_LQ_fits_unweighted.csv
